In [ ]:
import plotly.express as px
import polars as pl

from fryer import all as fryer

pl.Config.set_tbl_rows(100)

# Road Accidents from 1979 to Latest Published Year

### UK Gov - Dept for Transport

[Source](https://www.data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data)

[Guidance](https://www.gov.uk/guidance/road-accident-and-safety-statistics-guidance)

**Load a dictionary of road accident data**

keys are the dataset names below and values are the corresponding polars dataframes

- collision
- caualties
- vehicles

Casualties and vehicles can be joined via a zero or many to one relationship with the collision dataframe using the `accident_index` column

Additionally, `vehicle_reference` in the casualties and vehicles table will relate a casualty to a particular vehicle involved in the matching accident

[Local Authority borders](https://geoportal.statistics.gov.uk/search?q=BDY_LAD%202024&sort=Title%7Ctitle%7Casc)

In [ ]:
dfs = fryer.data.uk_gov_dept_for_transport_road_accident.read()
gdf = fryer.data.ons_local_authority_district_boundaries.read()
collision, casualty, vehicle = dfs.values()

for name, df in dfs.items():
    print(name, "\nrows:", f"{df.shape[0]:,}", "\ncolumns:", df.shape[1], "\n")

In [ ]:
data_without_ons_location = collision.filter(
    pl.col("latitude").is_null(),
    pl.col("longitude").is_null(),
    pl.col("local_authority_ons_district").is_null(),
)

print(
    "rows without latitude, longitude or ONS local authority location after deriving:",
    f"{data_without_ons_location.shape[0]:,}",
    f"{(data_without_ons_location.shape[0] / collision.shape[0]):.2%}",
    sep="\n",
)

data_without_any_location = collision.filter(
    pl.col("latitude").is_null(),
    pl.col("longitude").is_null(),
    pl.col("local_authority_ons_district").is_null(),
    pl.col("local_authority_district").is_null(),
)

print(
    "rows without latitude, longitude or ONS local authority or old local authority location after deriving:",
    f"{data_without_any_location.shape[0]:,}",
    f"{(data_without_any_location.shape[0] / collision.shape[0]):.2%}",
    sep="\n",
)

In [ ]:
px.bar(
    (
        data_without_ons_location.group_by(
            [
                pl.col("accident_year").alias("accident_year_int"),
                pl.col("accident_year").cast(pl.String),
                pl.col("accident_severity"),
            ],
        )
        .agg(pl.col("accident_index").count().alias("count"))
        .sort("accident_year_int")
    ),
    x="accident_year",
    y="count",
    color="accident_severity",
    barmode="group",
)